# Profiling
Sources:
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html
https://perso.crans.org/besson/publis/notebooks/Profiling_in_a_Jupyter_notebook.html

- `%time`: Time the execution of a single statement
- `%timeit`: Time repeated execution of a single statement for more accuracy
- `%prun`: Run code with the profiler
- `%lprun`: Run code with the line-by-line profiler
- `%memit`: Measure the memory use of a single statement
- `%mprun`: Run code with the line-by-line memory profiler -> Requires the code to be defined in a file!

## Cython magic options
https://ipython.org/ipython-doc/2/config/extensions/cythonmagic.html

```
%cython [-c COMPILE_ARGS] [--link-args LINK_ARGS] [-l LIB] [-n NAME] [-L dir] [-I INCLUDE] [-+] [-f] [-a]
```

In [1]:
%load_ext cython
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
import numpy as np

## Python profiling

In [3]:
def recip_square(i):
    return 1./i**2

def approx_pi(n=10000000):
    val = 0.
    for k in range(1,n+1):
        val += recip_square(k)
    return (6 * val)**.5

In [4]:
%time approx_pi()

CPU times: user 2.48 s, sys: 0 ns, total: 2.48 s
Wall time: 2.48 s


3.1415925580959025

In [5]:
%timeit approx_pi()

2.59 s ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%prun approx_pi()

         10000004 function calls in 3.237 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10000000    2.073    0.000    2.073    0.000 <ipython-input-3-83cbd44cb644>:1(recip_square)
        1    1.164    1.164    3.237    3.237 <ipython-input-3-83cbd44cb644>:4(approx_pi)
        1    0.000    0.000    3.237    3.237 {built-in method builtins.exec}
        1    0.000    0.000    3.237    3.237 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [7]:
%lprun -f approx_pi approx_pi()

Timer unit: 1e-06 s

Total time: 7.91809 s
File: <ipython-input-3-83cbd44cb644>
Function: approx_pi at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def approx_pi(n=10000000):
     5         1          2.0      2.0      0.0      val = 0.
     6  10000001    2191657.0      0.2     27.7      for k in range(1,n+1):
     7  10000000    5726426.0      0.6     72.3          val += recip_square(k)
     8         1          3.0      3.0      0.0      return (6 * val)**.5

In [8]:
%mprun approx_pi()

In [9]:
%mprun -f approx_pi approx_pi()

ERROR: Could not find file <ipython-input-3-83cbd44cb644>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.



## Cython profiling

In [10]:
%%cython -3 --annotate 
# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

def recip_square2(int i):
    return 1./i**2

def approx_pi2(int n=10000000):
    cdef double val = 0.
    cdef int k
    for k in range(1, n + 1):
        val += recip_square2(k)
    return (6 * val)**.5


In [11]:
%lprun approx_pi2()

Timer unit: 1e-06 s

In [12]:
%lprun -f approx_pi2 approx_pi2()

Timer unit: 1e-06 s

Total time: 2.96282 s
File: /home/devel/.cache/ipython/cython/_cython_magic_e6629e36c41e1c11ad663f499844b94e.pyx
Function: approx_pi2 at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def approx_pi2(int n=10000000):
    10         1          3.0      3.0      0.0      cdef double val = 0.
    11                                               cdef int k
    12         1          1.0      1.0      0.0      for k in range(1, n + 1):
    13  10000000    2962815.0      0.3    100.0          val += recip_square2(k)
    14         1          2.0      2.0      0.0      return (6 * val)**.5

In [13]:
%%cython -3 --annotate --cplus --force
# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

from libc cimport math as cmath
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)
cdef double recip_square3(int i):
    return (<double> 1)/cmath.pow(i, 2)

cpdef double approx_pi3(int n=10000000):
    cdef double val = 0.
    cdef int k
    for k in range(1, n + 1):
        val += recip_square3(k)
    return (6 * val)**.5

# NECESSARY SO THAT THE LINE-PROFILER FINDS THE CORRECT FCN
# WHEN PROFILING A CPDEF FCN! https://stackoverflow.com/a/57599927/11603367
def approx_pi3_sentinel():
    pass

In [14]:
%lprun -f approx_pi3 approx_pi3()

Timer unit: 1e-06 s

Total time: 2.3576 s
File: /home/devel/.cache/ipython/cython/_cython_magic_6a432915fa1b680f9b505a96bd02d770.pyx
Function: approx_pi3 at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           cpdef double approx_pi3(int n=10000000):
    17         1          1.0      1.0      0.0      cdef double val = 0.
    18                                               cdef int k
    19         1          1.0      1.0      0.0      for k in range(1, n + 1):
    20  10000000    2357599.0      0.2    100.0          val += recip_square3(k)
    21         1          2.0      2.0      0.0      return (6 * val)**.5
    22                                           
    23                                           # NECESSARY SO THAT THE LINE-PROFILER FINDS THE CORRECT FCN
    24                                           # WHEN PROFILING A CPDEF FCN! https://stackoverflow.com/a/57599927/11603367
    25                   

In [20]:
# import pstats, cProfile
# import calc_pi
# cProfile.runctx("calc_pi.approx_pi()", globals(), locals(), "Profile.prof")
# s = pstats.Stats("Profile.prof")
# s.strip_dirs().sort_stats("time").print_stats()